# Coreference Resolution

In this problem set, you will venture into the challenging NLP task of **coreference resolution**. You will:

* Implement a simple rule-based system that achieve results which are surprisingly difficult to beat.
* Get acquainted with the trickiness of evaluating coref systems, and the current solutions in the field.
* Experiment with two neural approaches for coref to be implemented in [PyTorch](pytorch.org):
  * A feedforward network that only looks at boolean mention-pair features
  * A fully-neural architecture with embeddings all the way down
* Get a glimpse at **domain adaptation** in the wild, by trying to run a system trained on news against a narrative corpus and vice-versa.


# 0. Setup

In order to develop this assignment, you will need [python 3.6](https://www.python.org/downloads/) and the following libraries. Most if not all of these are part of [anaconda](https://www.continuum.io/downloads), so a good starting point would be to install that.

* [jupyter](http://jupyter.readthedocs.org/en/latest/install.html)
* [numpy](https://docs.scipy.org/doc/numpy/user/install.html)
* [nosetests](https://nose.readthedocs.org/en/latest/)
* [pytorch](https://pytorch.org/)
* [scikit-learn](http://scikit-learn.org/)
* [xmltodict](https://github.com/martinblech/xmltodict#ok-how-do-i-get-it)

Here is some help on installing packages in python: https://packaging.python.org/installing/. You can use pip --user to install locally without sudo.

## About this assignment

* This is a Jupyter notebook. You can execute cell blocks by pressing control+enter, or execute and move to the next using shift+enter, or execute and open a new cell below using alt+enter.
* Most of your coding will be in the python source files in the directory `gtnlplib`.
* The directory `tests` contains unit tests that will be used to grade your assignment, using `nosetests`. You should run them as you work on the assignment to see that you're on the right track. You are free to look at their source code, if that helps -- though most of the relevant code is also here in this notebook. Learn more about running unit tests [here](http://pythontesting.net/framework/nose/nose-introduction/).
* You may want to add more tests, but that is completely optional.
* **To submit this assignment, run the script `make-submission.sh`, and submit the tarball `pset4-submission.tgz` on Canvas.**

In [1]:
import nose
import numpy as np
import os
import pickle
import sklearn
import xmltodict
from collections import Counter, defaultdict
from nltk.tag import pos_tag

import torch
from torch.nn import functional as F
import torch.optim as optim

# no more reload cells!
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
print('My library versions')

print('numpy: {}'.format(np.__version__))
print('xmltodict: {}'.format(xmltodict.__version__))
print('nose: {}'.format(nose.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('torch: {}'.format(torch.__version__)) # this TA uses a Windows machine, so YMMV

My library versions
numpy: 1.14.0
xmltodict: 0.11.0
nose: 1.3.7
sklearn: 0.19.1
torch: 0.3.0b0+591e73e


To test whether your libraries are the right version, run:

`nosetests tests/test_environment.py`

In [3]:
# use ! to run shell commands in notebook
! nosetests tests/test_environment.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [4]:
from gtnlplib import coref, coref_rules, coref_features, coref_learning, neural_net, utils

# constants for notebook use
ETA_0 = 0.01

# Part 1: Exploring the data

The core data is in the form of "markables", or "referring expressions", which refer to token sequences that can participate in coreference relations.

Each markable is a namedtuple with five elements:
- ```string```, which is a list of tokens
- ```entity```, which defines the ground truth assignments
- ```start_token```, the index of the first token in the markable with respect to the entire document
- ```end_token```, one plus the index of the last token in the markable
- ```tags```, POS tags corresponding to the tokens in ```string``` which will remain NULL for now

The ```read_data``` function also returns a list of tokens.
You can use this to incorporate the linguistic context around each markable.

### Loading the dataset

For most of this problem set, we will explore a dataset of articles from the Wall Street Journal (WSJ) extracted and annotated from the Penn Treebank (PTB).

In [5]:
dv_dir = os.path.join('data','wsj','dev')
tr_dir = os.path.join('data','wsj','train')
te_dir = os.path.join('data','wsj','test') # all markables here are annotated as the same entity

In [6]:
markables, words = coref.read_data('06_wsj_0051.sty',basedir=tr_dir)

In [7]:
print('Markable object:', markables[0])
print('Words for markable extracted from text:', words[markables[0].start_token:markables[0].end_token])

Markable object: Markable(string=['Fujitsu', 'Ltd.'], entity='set_3082', start_token=0, end_token=2, tags=['NULL', 'NULL'])
Words for markable extracted from text: ['Fujitsu', 'Ltd.']


**Deliverable 1.1**: Write a function that returns all the markable **strings** associated with a given entity. Specifically, fill in the function `get_markables_for_entity()` in `coref.py`.
(4650: 1 point; 7650: 0.5 points)

* **Test:** `tests\test_coref.py:test_get_markables_d1_1()`

In [8]:
sorted(coref.get_markables_for_entity(markables,'set_3082'))

['Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 'Fujitsu',
 "Fujitsu , Japan 's No. 1 computer maker",
 'Fujitsu Ltd.',
 'It',
 'The company',
 'The company',
 'We',
 'his company',
 'his company',
 'it',
 'it',
 'it',
 'it',
 'it',
 'it',
 'its',
 'its',
 'the company']

**Deliverable 1.2** Write a function that takes as input a string, and returns a list of distances to the most recent ground truth antecedent for every time the (case-insensitive) input string appears. For example, if the input is "they", it should make a list with one element for each time the word "they" appears in the list of markables. Each element should be the distance of the word "they" to the nearest previous mention of the entity that "they" references.

Fill in the function `get_distances()` in `coref.py`. If the input string is not anaphoric, the distance should be zero. Note that input strings may contain spaces. You may use any other function in `coref.py` to help you.
(4650: 1 point; 7650: 0.5 points)

* **Test:** `tests\test_coref.py:test_get_antecedents_d1_2()`

In [9]:
coref.get_distances(markables,'they')

[1, 1, 1, 2, 2]

Now let's compare the typical distances for various mention types.

You can see the most frequent mention types by using the `Counter` class.

In [10]:
Counter([' '.join(markable.string) for markable in markables]).most_common(5)

[('it', 9), ('Fujitsu', 7), ('Japan', 5), ('they', 5), ('NEC', 4)]

In [11]:
coref.get_distances(markables, 'Fujitsu')

[15, 8, 49, 12, 7, 4, 2]

In [12]:
coref.get_distances(markables, 'the company')

[4, 4, 6]

In [13]:
coref.get_distances(markables, 'it') # there are 10 because our counter was case-sensitive

[1, 2, 2, 1, 6, 1, 6, 2, 6, 1]

# 2. Rule-based coreference resolution

We have written a simple coreference classifier, which predicts that each markable is linked to the most recent antecedent which is an exact string match.

The code block below applies this method to the dev set.

In [14]:
exact_matcher = coref_rules.make_resolver(coref_rules.exact_match)

The code above has two pieces:

- ```coref_rules.exact_match()``` is a function that takes two markables, and returns `True` iff they are an exact (case-insensitive) string match
- ```make_resolver()``` is a function that takes a matching function, and returns a function that computes an antecedent list for a list of markables.

Let's run it.

In [15]:
ant_exact = exact_matcher(markables)
print(ant_exact[:20])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


The output is a list of antecedent numbers, $c_i$. 
When $c_i = i$, the markable $i$ has no antecedent: it is the first mention of its entity. In this case, all first 20 mentions are new and don't have antecedents. You can try and modify the cell to look further down the list to see if there are ever actual matches, which we know should occur due to the output of cell 5.

We can test whether these predictions are correct by comparing against the key.

In [16]:
ant_true = coref.get_true_antecedents(markables)

In [17]:
num_correct = sum([c_true == c_predict for c_true, c_predict in zip(ant_true, ant_exact)])
acc = num_correct / len(markables)
print(f'correct: {num_correct}\taccuracy: {acc:.3f}')

correct: 128	accuracy: 0.660


## Evaluation

Coreference can be evaluated in terms of recall, precision, and F-measure. Here is how we will define these terms:

- **True positive**: The system predicts $\hat{c}_i < i$, and $\hat{c}_i$ and $i$ are references to the same entity.
- **False positive**: The system predicts $\hat{c}_i < i$, but $\hat{c}_i$ and $i$ are not references to the same entity.
- **False negative**: There exists some $c_i < i$ such that $c_i$ and $i$ are references to the same entity, but the system predicts either $\hat{c}_i = i$, or some $\hat{c}_i$ which is not really a reference to the same entity that $i$ references.
- Recall = $\frac{tp}{tp + fn}$
- Precision = $\frac{tp}{tp + fp}$
- F-measure = $\frac{2RP}{R+P}$

A couple of things to notice here:

- There is no reward for correctly identifying a markable as non-anaphoric (not having any antecedent), but you do avoid committing a false positive by doing this.
- You cannot compute the evaluation by directly matching the predicted antecedents to the true antecedents. Suppose the truth is $a \leftarrow b, b \leftarrow c$, but the system predicts $a \leftarrow b, a \leftarrow c$: the system should receive two true positives, since $a$ and $c$ are references to the same entity in the ground truth.

**Deliverable 2.1** Implement `get_tp()`, `get_fp()`, and `get_fn()` in `coref.py`. You will want to use the function `coref.get_entities()`.
(1 point)

* **Test:** `tests\test_coref.py:test_recall_d2_1(), test_precision_d2_1(), test_fmeasure_d2_1()`

**NOTE!** You **must** successfully complete this deliverable. Otherwise, some of the unit tests won't work and you won't be able to complete the rest of the assignment.

In [18]:
f,r,p = coref.evaluate_f(exact_matcher, markables)
print(f'{f:.4f}\t{r:.4f}\t{p:.4f}')

0.5231	0.4096	0.7234


In [19]:
all_markables, all_words = coref.read_dataset(tr_dir)

In [20]:
coref.eval_on_dataset(exact_matcher, all_markables);

F: 0.5452	R: 0.4130	P:0.8018


Before optimizing on this simple F-measure (sometimes called F1) and its components, one should be aware that in the real world coreference is evaluated over three other metrics, namely $B^3$, **`CEAF`**, and **`MUC`**. We have implemented them for you, and will check our performance on them from time to time. You can read more about them [here](http://www.anthology.aclweb.org/W/W10/W10-4305.pdf).

In [21]:
def coref_metrics(matcher, dataset):
    ants = [matcher(m) for m in dataset]
    b3, ceaf, muc = coref.evaluate_bcm(dataset, ants)
    avg_f1 = np.average([b3, ceaf, muc])
    print(f'B-Cubed: {b3:.4f}\tCEAF: {ceaf:.4f}\tMUC: {muc:.4f}\tAverage: {avg_f1:.4f}')

In [22]:
coref_metrics(exact_matcher, all_markables); #"Average" is the commonly used main metric in state-of-the-art systems.

B-Cubed: 0.5756	CEAF: 0.4551	MUC: 0.5605	Average: 0.5304


The reasons for having multiple measures for coref evaluation is manyfold. One of them, discussed in the paper linked to above, has to do with the pre-resolution task of *identifying markables*. Since we're only working with pre-extracted markables, that needn't worry us.

The other reason is that different perspectives on coreference matching are equally plausible - we can focus on single correct predictions, or finding the correct clusters for each entity , for example. Each of these is "gameable" by different trivial classifiers.

**Deliverable 2.2** To witness this problem, you will implement `coref_rules.singleton_matcher()`, which produces an assignment where each markable has its own entity, and `coref_rules.full_cluster_matcher()`, which assigns all markables to the same entity. Running the metrics against them will demonstrate the problem.
(0.5 points)

* **Test:** `tests\test_coref.py:test_singleton_matcher_d2_2(), test_full_cluster_matcher_d2_2()`

In [23]:
singleton_resolver = coref_rules.make_resolver(coref_rules.singleton_matcher)
coref_metrics(singleton_resolver, all_markables);

B-Cubed: 0.3411	CEAF: 0.0012	MUC: 0.0000	Average: 0.1141


MUC has an inherent problem evaluating singleton entities. $B^3$, on the other hand, is extra-generous with them.

In [24]:
full_cluster_resolver = coref_rules.make_resolver(coref_rules.full_cluster_matcher)
coref_metrics(full_cluster_resolver, all_markables);

B-Cubed: 0.0739	CEAF: 0.0304	MUC: 0.5552	Average: 0.2199


In this case MUC, which is focused on detecting incompatible clusters, is fairly comfortable with the fact that there's only one predicted cluster.
CEAF, a metric which gives low precision very easily, is difficult to score high on.

## Increasing precision

The `exact_match()` function matches everything, including pronouns. This can lead to mistakes:

"Umashanthi ate pizza until she was full. Parvati kept eating until she had a stomach ache."

In this example, both pronouns likely refer to the names that immediately precede them, and not to each other.

**Deliverable 2.3** The file `coref_rules.py` contains the signature for a function `exact_match_no_pronoun()`, which solves this problem by only predicting matches between markables that are not pronouns. Implement and test this function. For now, you may use the list of pronouns provided in the code file `coref_rules.py`.
(0.25 points)

* **Test:** `tests\test_coref.py:test_match_nopro_d2_3(), tests\test_coref.py:test_match_nopro_f1_d2_3()`

In [25]:
no_pro_matcher = coref_rules.make_resolver(coref_rules.exact_match_no_pronouns)

In [26]:
f,r,p = coref.eval_on_dataset(no_pro_matcher,all_markables);

F: 0.4551	R: 0.3028	P:0.9158


In [27]:
coref_metrics(no_pro_matcher, all_markables);

B-Cubed: 0.5678	CEAF: 0.4269	MUC: 0.4568	Average: 0.4839


Precision has increased, but recall decreased, dragging down the overall F-measure as well as our favorite metrics.

## Increasing recall

Our current matcher is very conservative. Let's try to increase recall. One solution is match on the **head word** of each markable. 

As you know, in a CFG parse, the head word is defined by a set of rules: for example, the head of a determiner-noun construction is the noun. In a dependency parse, the head word would be the root of the subtree governing the markable span. But this assumes that the markables correspond to syntactic constituents or dependency subtrees. This is not guaranteed to be true - particularly when there are parsing errors.

**Deliverable 2.4** Let's start with a much simpler head-finding heuristic: simply select the *last word* in the markable. This handles many cases - but as we will see, not all. To do this, implement the function `match_last_token()` in ```coref_rules.py```. This function should match all cases where the final tokens match.
(0.25 points)

* **Test:** `tests\test_coref.py:test_match_last_tok_d2_4()`

In [28]:
last_tok_matcher = coref_rules.make_resolver(coref_rules.match_last_token)

In [29]:
coref.eval_on_dataset(last_tok_matcher,all_markables);

F: 0.3994	R: 0.4385	P:0.3666


Recall is up, but precision is back down. To try to increase precision, let's add one more rule: two markables cannot coref if their spans overlap. This can happen with nested mentions, such as "(the president (of the United States))". Under our last-token rule, these two mentions would co-refer, but logically, overlapping markables cannot refer to the same entity. 

**Deliverable 2.5** Fill in the function `match_last_token_no_overlap()`, which should match any two markables that share the same last token, unless their spans overlap. Use the `start_token` and `end_token` members of each markable to determine whether they overlap. the final tokens match.
(0.25 points)

* **Test:** `tests\test_coref.py:test_match_no_overlap_f1_d2_5()`

In [30]:
mltno_matcher = coref_rules.make_resolver(coref_rules.match_last_token_no_overlap)
coref.eval_on_dataset(mltno_matcher,all_markables);

F: 0.4911	R: 0.4965	P:0.4858


Both recall and precision increase. Why would recall increase? The restriction does not create any new coreference links, but it changes some incorrect links to correct links. This increases the number of true positives and reduces the number of false negatives.

In [31]:
coref_metrics(mltno_matcher, all_markables);

B-Cubed: 0.5571	CEAF: 0.4610	MUC: 0.5473	Average: 0.5218


Almost back to the results from the exact matcher.

## Error analysis

To see whether we can do even better, let's try some error analysis on a specific file.

In [32]:
# predicted antecedent series
markables_17, _ = coref.read_data('17_wsj_0072.sty',basedir=tr_dir)
ant = coref_rules.make_resolver(coref_rules.match_last_token_no_overlap)(markables_17)

In [33]:
# let's look at large entities
m2e, e2m = coref.markables_to_entities(markables_17,ant)
big_entities = [ent for ent, vals in e2m.items() if len(vals) > 10]

In [34]:
for entity in big_entities:
    print(f'Entity {entity}: {len(e2m[entity])} mentions')
    print([' '.join(markables_17[idx].string) for idx in e2m[entity]])
    print()

Entity 8: 11 mentions
['Fed', 'Kansas City Fed', 'the Fed', 'the Fed', 'The Fed', 'The report from the Fed', 'the Fed', 'The Philadelphia Fed', 'Fed', 'Fed', 'regional Fed']



## Incorporating parts of speech

One clear mistake is that we are matching ''Kansas City Fed'' to ''The Philadelphia Fed'' and other ''Fed''s. The last token heuristic is the culprit: in this case, the first token is a key disambiguator. Let's try a more syntactically-motivated approach. 

Instead of matching the last token (low precision) or matching on all tokens (low recall), let's try matching on all *content* words. Let's start by including only the following grammatical categories:

- Nouns (proper, common, singular, plural)
- Pronouns (including possessive)
- Adjectives (including comparative and superlative)
- Cardinal numbers

To get these categories, we can call `read_dataset()` again with the optional `tagger` argument, a part of speech tagger. We'll use NLTK for this project, which has a structured perceptron tagger on the [PTB tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html). 

In [35]:
all_markables, _ = coref.read_dataset(tr_dir, tagger=pos_tag)
all_markables_dev, all_words_dev = coref.read_dataset(dv_dir, tagger=pos_tag)
all_markables_te, all_words_test = coref.read_dataset(te_dir, tagger=pos_tag)

In [36]:
all_markables[1][1]

Markable(string=['a', 'trade', 'deficit', 'of', '$', '101', 'million'], entity='set_972', start_token=3, end_token=10, tags=['DT', 'NN', 'NN', 'IN', '$', 'CD', 'CD'])

As you can see, the markables now have the `tags` member populated with the part-of-speech tags for each token in the `string` field.

**Deliverable 2.6** Now implement a new matcher, `coref_rules.match_on_content()`. Your code should match $m_a$ and $m_i$ iff all content words are identical. It should also enforce the "no overlap" restriction defined above.
(0.5 points)

* **Test:** `tests\test_coref.py:test_match_content_f1_d2_6()`

In [37]:
content_matcher = coref_rules.make_resolver(coref_rules.match_on_content)
coref.eval_on_dataset(content_matcher, all_markables);

F: 0.5565	R: 0.4374	P:0.7647


In [38]:
coref_metrics(content_matcher, all_markables);

B-Cubed: 0.5872	CEAF: 0.4725	MUC: 0.5727	Average: 0.5441


Finally getting some headway on those metrics!

**Deliverable 2.7** Run the code blocks below to output predictions for the dev and test data.
(0.5 points)

* **Test:** `tests\test_coref.py:test_dev_acc_f1_d2_7(), test_test_acc_f1_d2_7()`

In [ ]:
# run once, then remove/comment out cell after dir is created
!mkdir predictions

In [39]:
coref.write_predictions(coref_rules.make_resolver(coref_rules.match_on_content),
                        all_markables_dev,
                        'predictions/rules-dev.preds')

In [40]:
f,r,p = coref.eval_predictions('predictions/rules-dev.preds',all_markables_dev);

F: 0.5463	R: 0.3960	P:0.8806


In [41]:
coref_metrics(content_matcher, all_markables_dev);

B-Cubed: 0.5768	CEAF: 0.4045	MUC: 0.5463	Average: 0.5092


In [42]:
coref.write_predictions(coref_rules.make_resolver(coref_rules.match_on_content),
                        all_markables_te,
                        'predictions/rules-test.preds')

In [43]:
# students can't run this (well, they can but it'll match against a full-cluster match)
coref.eval_predictions('predictions/rules-test.preds', all_markables_te);
coref_metrics(content_matcher, all_markables_te);

F: 0.4991	R: 0.3811	P:0.7231
B-Cubed: 0.5522	CEAF: 0.4650	MUC: 0.5168	Average: 0.5113


# Part 3: Machine learning for coreference resolution

You will now implement coreference resolution using the mention-ranking model. Let's start by implementing some features.

**Deliverable 3.1** Implement `coref_features.minimal_features`, using the rules you wrote from `coref_rules.` This should be a function that takes a list of markables, and indices for two mentions, and returns a dict with features and counts. Include the following features:

- `exact-match`
- `last-token-match`
- `content-match`
- `crossover`: value of 1 iff the mentions overlap
- `new-entity`: value of 1 iff i=j

For the first four features, you should call your code from coref_rules directly.
(0.25 points)

* **Test:** `tests\test_coref.test_minimal_features_d3_1()`

In [44]:
min_features = ['exact-match', 'last-token-match', 'content-match', 'crossover', 'new-entity']
for i, markable in enumerate(all_markables[1][:15]):
    print(i, markable)

0 Markable(string=['South', 'Korea'], entity='set_971', start_token=0, end_token=2, tags=['NNP', 'NNP'])
1 Markable(string=['a', 'trade', 'deficit', 'of', '$', '101', 'million'], entity='set_972', start_token=3, end_token=10, tags=['DT', 'NN', 'NN', 'IN', '$', 'CD', 'CD'])
2 Markable(string=['October'], entity='set_973', start_token=11, end_token=12, tags=['NNP'])
3 Markable(string=['the', 'country', "'s", 'economic', 'sluggishness'], entity='set_974', start_token=14, end_token=19, tags=['DT', 'NN', 'POS', 'JJ', 'NN'])
4 Markable(string=['country'], entity='set_971', start_token=15, end_token=16, tags=['NN'])
5 Markable(string=['government', 'figures'], entity='set_975', start_token=22, end_token=24, tags=['NN', 'NNS'])
6 Markable(string=['Wednesday'], entity='set_976', start_token=25, end_token=26, tags=['NNP'])
7 Markable(string=['Preliminary', 'tallies'], entity='set_977', start_token=27, end_token=29, tags=['JJ', 'NNS'])
8 Markable(string=['the', 'Trade', 'and', 'Industry', 'Minist

In [45]:
print(coref_features.minimal_features(all_markables[1],0,1))
print(coref_features.minimal_features(all_markables[1],0,13))
print(coref_features.minimal_features(all_markables[1],13,14))
print(coref_features.minimal_features(all_markables[1],6,6))
print(coref_features.minimal_features(all_markables[1],2,10))

defaultdict(<class 'float'>, {})
defaultdict(<class 'float'>, {'exact-match': 1.0, 'last-token-match': 1.0, 'content-match': 1.0})
defaultdict(<class 'float'>, {'crossover': 1.0})
defaultdict(<class 'float'>, {'new-entity': 1.0})
defaultdict(<class 'float'>, {'exact-match': 1.0, 'last-token-match': 1.0, 'content-match': 1.0})


**Deliverable 3.2** You will now use these features in a simple feedforward neural net. Using pytorch, implement the `__init__()` and `forward()` functions in `coref_learning.FFCoref` which will be composed of two linear layers separated by a tanh nonlinearity, producing a score for each possible antecedent.

Later we will use this scoring function to select the most probable antecendent.
(1 point)

* **Test:** `tests\test_neural_coref.py:test_ffcoref_d3_2()`

In [46]:
COREF_FF_HIDDEN = 5 # dimension for hidden layer

In [47]:
torch.manual_seed(1984) # DO NOT CHANGE
coref_ff = coref_learning.FFCoref(min_features, COREF_FF_HIDDEN)

# scores for single mention pairs, no backprop
print(coref_ff(coref_features.minimal_features(all_markables[1],0,1)))
print(coref_ff(coref_features.minimal_features(all_markables[1],0,13)))
print(coref_ff(coref_features.minimal_features(all_markables[1],2,10)))

Variable containing:
-0.1148
[torch.FloatTensor of size 1]

Variable containing:
-0.3763
[torch.FloatTensor of size 1]

Variable containing:
-0.3763
[torch.FloatTensor of size 1]



**Deliverable 3.3** Given a markable and all its possible antecedents, we now wish to feed all the scores into a softmax layer and attempt the highest possible sum of likelihoods for antecedents representing the correct entity. Implement `FFCoref.score_instance()` in order to do so.
(0.5 points)

* **Test:** `tests\test_neural_coref.py:test_ffcoref_score_instance_d3_3()`

In [48]:
i_scores = coref_ff.score_instance(all_markables[1], coref_features.minimal_features, 13)
print(i_scores)

Variable containing:

Columns 0 to 9 
-0.3763 -0.1148 -0.1148 -0.1148 -0.1148 -0.1148 -0.1148 -0.1148 -0.1148 -0.1148

Columns 10 to 13 
-0.1148 -0.1148 -0.1148 -0.0424
[torch.FloatTensor of size 1x14]



In inference time, all we need is to use the above function and report the antecedent for each markable. In training time, we will use an objective based on the **hinge margin-loss** function. Our variant will require the highest-scoring false candidate (according to the true entity annotations) to score lower than the highest-scoring true candidate *by a margin*:

$$L_{m_i} = \{ \text{max}_{a:m_a\notin A(m_i)} s(m_i,m_a) + M - \text{max}_{a:m_a\in A(m_i)} s(m_i,m_a) \}_{+}$$

Where $s$ is the score from the previous deliverable, $A(m)$ denotes the set of true antecedents for $m$, $M$ is our margin, and the $+$ subscript indicates that negative values are replaced by $0$ (since this is a hinge loss).

**Deliverable 3.4**
Implement the helper function `FFCoref.instance_top_scores()` which supplies the arguments for this loss function.
**Note** the special cases where:
- Only true candidates exist (we're in the first cluster) - the trainer will have to skip these. Return `None`s.
- Only false candidates exist - this actually means we're in a new cluster.

(0.5 points)


* **Test:** `tests\test_neural_coref.py:test_ffcoref_score_instance_top_scores_d3_4()`

In [49]:
best_true_score, best_false_score = coref_ff.instance_top_scores(all_markables[1], coref_features.minimal_features, 13, 4)
print(torch.cat([best_true_score, best_false_score], 0))

Variable containing:
-0.1148
-0.0424
[torch.FloatTensor of size 2]



Looks like we're ready to train our classifier!

In [51]:
torch.manual_seed(1984) # DO NOT CHANGE
coref_ff = coref_learning.FFCoref(min_features, COREF_FF_HIDDEN)
optimizer = optim.SGD(coref_ff.parameters(), lr=ETA_0)
coref_learning.train(coref_ff, optimizer, all_markables, coref_features.minimal_features, margin=1.0, epochs=2)

Loss = 0.9191775133819172
Loss = 0.9106459462424766


In [52]:
# training set results
coref_learning.evaluate(coref_ff, all_markables, coref_features.minimal_features)

F: 0.5310	R: 0.4176	P:0.7287


In [53]:
# dev set
coref_learning.evaluate(coref_ff, all_markables_dev, coref_features.minimal_features)

F: 0.5505	R: 0.4027	P:0.8696


In [54]:
# standard metrics on training set
ff_matcher = coref_learning.make_resolver(coref_features.minimal_features, coref_ff)
coref_metrics(ff_matcher, all_markables);

B-Cubed: 0.5869	CEAF: 0.4718	MUC: 0.5723	Average: 0.5436


In [55]:
#test set, students can't run
coref_metrics(ff_matcher, all_markables_te);

B-Cubed: 0.5522	CEAF: 0.4650	MUC: 0.5168	Average: 0.5113


**Deliverable 3.5** We can add more features to try and better capture relations between markables.

Implement distance features in `coref_features.distance_features()`, measuring the mention distance and the token distance. Specifically:

- **Mention distance** is number of intervening mentions between i and j, $i-j$.
- **Token distance** is number of tokens between the start of i and the end of j.

These should be binary features, up to a maximum distance of 10 for tokens / 5 for mentions, with the final feature indicating distance of 10/5 and above, respectively. The desired behavior is shown below.
(0.5 points)

* **Test:** `tests\test_coref.py:test_distance_features_d3_5()`

In [56]:
for i, markable_i in enumerate(all_markables[1][:4]):
    print(i, markable_i)

0 Markable(string=['South', 'Korea'], entity='set_971', start_token=0, end_token=2, tags=['NNP', 'NNP'])
1 Markable(string=['a', 'trade', 'deficit', 'of', '$', '101', 'million'], entity='set_972', start_token=3, end_token=10, tags=['DT', 'NN', 'NN', 'IN', '$', 'CD', 'CD'])
2 Markable(string=['October'], entity='set_973', start_token=11, end_token=12, tags=['NNP'])
3 Markable(string=['the', 'country', "'s", 'economic', 'sluggishness'], entity='set_974', start_token=14, end_token=19, tags=['DT', 'NN', 'POS', 'JJ', 'NN'])


In [57]:
print(coref_features.distance_features(all_markables[1],0,0))
print(coref_features.distance_features(all_markables[1],0,1))
print(coref_features.distance_features(all_markables[1],0,2))
print(coref_features.distance_features(all_markables[1],1,3))
print(coref_features.distance_features(all_markables[1],0,30))

defaultdict(<class 'float'>, {})
defaultdict(<class 'float'>, {'mention-distance-1': 1, 'token-distance-1': 1})
defaultdict(<class 'float'>, {'mention-distance-2': 1, 'token-distance-9': 1})
defaultdict(<class 'float'>, {'mention-distance-2': 1, 'token-distance-4': 1})
defaultdict(<class 'float'>, {'mention-distance-5': 1, 'token-distance-10': 1})


**Deliverable 3.6** Implement `coref_features.make_feature_union()`, which should take a list of feature functions, and return a function that computes the union of all features in the list. You can assume the feature functions don't use the same name for any feature.
(0.5 points)

* **Test:** `tests\test_coref.py:test_feature_union_d3_6()`

In [58]:
joint_feats = coref_features.make_feature_union([coref_features.minimal_features,
                                                 coref_features.distance_features])

In [59]:
print(joint_feats(all_markables[1],1,3))
print(joint_feats(all_markables[1],0,3))
print(joint_feats(all_markables[1],0,7))
print(joint_feats(all_markables[1],10,10))

defaultdict(<class 'float'>, {'mention-distance-2': 1, 'token-distance-4': 1})
defaultdict(<class 'float'>, {'mention-distance-3': 1, 'token-distance-10': 1})
defaultdict(<class 'float'>, {'mention-distance-5': 1, 'token-distance-10': 1})
defaultdict(<class 'float'>, {'new-entity': 1.0})


In [60]:
min_features_and_distances = min_features\
                                   + [f'mention-distance-{i}' for i in range(1,6)]\
                                   + [f'token-distance-{i}' for i in range(1,11)]

In [61]:
torch.manual_seed(1984) # DO NOT CHANGE
coref_ff_w_distances = coref_learning.FFCoref(min_features_and_distances, 50) # we need more hidden units now
optimizer = optim.SGD(coref_ff_w_distances.parameters(), lr=ETA_0)
coref_learning.train(coref_ff_w_distances, optimizer, all_markables, joint_feats, epochs=2)

Loss = 0.9255589622700262
Loss = 0.8616410730355986


In [62]:
coref_learning.evaluate(coref_ff_w_distances, all_markables, joint_feats)

F: 0.5424	R: 0.4118	P:0.7942


In [63]:
ff_w_dist_matcher = coref_learning.make_resolver(joint_feats, coref_ff_w_distances)
coref_metrics(ff_w_dist_matcher, all_markables);

B-Cubed: 0.5780	CEAF: 0.4572	MUC: 0.5623	Average: 0.5325


Note that our basic F metric got slightly better, while the average standard metric got slightly worse due to reduced MUC and CEAF.

In [64]:
coref.write_predictions(ff_w_dist_matcher,
                        all_markables_dev,
                        'predictions/ff-dev.preds')
coref.eval_predictions('predictions/ff-dev.preds', all_markables_dev);

F: 0.5213	R: 0.3691	P:0.8871


In [65]:
coref_metrics(ff_w_dist_matcher, all_markables_dev);

B-Cubed: 0.5641	CEAF: 0.3940	MUC: 0.5308	Average: 0.4963


In [66]:
coref.write_predictions(ff_w_dist_matcher,
                        all_markables_te,
                        'predictions/ff-test.preds')

In [67]:
# students can't run this
coref.eval_predictions('predictions/ff-test.preds', all_markables_te);
coref_metrics(ff_w_dist_matcher, all_markables_te);

F: 0.4799	R: 0.3541	P:0.7443
B-Cubed: 0.5414	CEAF: 0.4356	MUC: 0.4908	Average: 0.4893


# Part 4: Sequential Text Represenation

In this section, we will find out whether neural representations of our text can help find coreferents.

The main idea is to run a bidirectional LSTM model, which you already have implemented from previous problem sets, and use the resulting hidden states to form representations of the markables. These will be fed into a feedforward classifier similar to the one from the previous section, except that the match features will also be embedded.

In [68]:
# Preparing the vocabulary for a word-to-index dictionary necessary for the initial embeddings table
vocab = set()
for doc in all_words + all_words_dev + all_words_test:
    vocab.update(doc)
vocab = sorted(list(vocab))
word_to_ix = {w:i for i,w in enumerate(vocab)}
print(len(vocab))

3530


**Deliverable 4.1**
The first part will be very similar to code from PS3, which you may reuse. Implement `neural_net.BiLSTMWordEmbedding` as a word  embedding lookup table followed by a bi-directional LSTM which runs on a text (here, the entire document) and outputs the hidden state from the LSTM as a contextual embedding for each word in it.
(1 point)

* **Test:** `tests\test_neural_coref.py:test_bilstm_embedding_d4_1()`

In [69]:
WORD_EMB_DIM = 64
WORD_LSTM_EMB_DIM = 128

In [70]:
torch.manual_seed(1984)
word_lstm = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.5)
embs = word_lstm(all_words[0])
print(' '.join(all_words[0][:17] + ['...']), '\n')
print(all_words[0][4], '\n', embs[4][0][:5])
print(all_words[0][13], '\n', embs[13][0][:5])

McDermott International Inc. said its Babcock & Wilcox unit completed the sale of its Bailey Controls Operations ... 

its 
 Variable containing:
-0.1957
-0.2839
 0.2615
-0.0888
-0.1205
[torch.FloatTensor of size 5]

its 
 Variable containing:
-0.1274
-0.1916
 0.2225
-0.1166
-0.0630
[torch.FloatTensor of size 5]



We see how the same word type (*its*) is assigned different embeddings based on its context in the document.

## Attention Model

Our markable embeddings will be trained using an **Attention Model** which accepts the embeddings for the words $\{w_i\}$ in a markable and outputs a single vector that "attends" to the single vectors according to what it believes is their importance.
This concept, [originally used](https://arxiv.org/abs/1409.0473) for sequence-to-sequence models such as Machine Translation, is applied for our task as yet another attempt to find the crucial part of a markable, like we did for the head-finding heuristic and for the content-word matching. While those were "hard" techniques, yes-or-no for each token, here we're applying a "soft" weighting that still assigns all the words in the text some significance.

Practically, our model will train a vector parameter of the same embedding size as the BiLSTM output, $\vec{u}$. Each word's embedding in the input span $\vec{e_i}$ will be multiplied (dot-product) with $\vec{u}$ to assign it a scalar weight, $a_i$. Finally each embedding will be multiplied by its normalized (softmaxed) weight $\alpha_i$, and the sum of these weighted vectors will be our markable's output embedding, $\vec{e_m}$:

![Attention equations](att.PNG "Attention equations")

**Deliverable 4.2**
Implement `neural_net.AttentionBasedMarkableEmbedding`.
(0.5 points)

* **Test:** `tests\test_neural_coref.py:test_embedding_attention_d4_2()`

In [71]:
torch.manual_seed(1984)
attn_layer = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
mark_embs = [attn_layer(embs, m) for m in all_markables[0]]
for j in [0, 1, 4]:
    print(all_markables[0][j].string, all_markables[0][j].entity, '\n', mark_embs[j][:3])

['McDermott', 'International', 'Inc.'] set_356 
 Variable containing:
-0.1441
-0.1028
-0.1215
[torch.FloatTensor of size 3]

['its'] set_356 
 Variable containing:
-0.1957
-0.2839
 0.2615
[torch.FloatTensor of size 3]

['its'] set_357 
 Variable containing:
-0.1274
-0.1916
 0.2225
[torch.FloatTensor of size 3]



We will score a markable pair based on the following features:

1. Each markable's attended embedding
1. A low-dimension embedding for each of the pairwise features we've extracted in the previous sections. Since they are boolean, each will have an embedding for its ''false'' state and one for its ''true'' state.

First, let's implement a quick extractor for positive-valued features from a mention pair.

In [72]:
def get_positive_feats(doc, i, a, feats=coref_features.minimal_features):
    return [k for k,v in feats(doc,i,a).items() if v > 0.0]

In [73]:
get_positive_feats(all_markables[1], 0, 13)

['exact-match', 'last-token-match', 'content-match']

Now we will concatenate all of these embeddings together and use them as input in a two-layer feedforward network (with ReLU nonlinearity) which will produce a scalar score for markable match.

**Deliverable 4.3**
Implement `__init__()` and `forward()` in `neural_net.SequentialScorer`.
(0.5 points)

* **Test:** `tests\test_neural_coref.py:test_sequential_scorer_d4_3()`

In [74]:
BOOLEAN_FEATURE_DIM = 6
SCORER_HIDDEN_DIM = 164

# starting with just one document
torch.manual_seed(1984)
word_lstm1 = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.5)
embs1 = word_lstm1(all_words[1])
attn_layer1 = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
mkbls1 = all_markables[1]
mark_embs1 = [attn_layer(embs1, m) for m in mkbls1]
scorer = neural_net.SequentialScorer(WORD_LSTM_EMB_DIM, min_features, BOOLEAN_FEATURE_DIM, SCORER_HIDDEN_DIM)
scorer(mark_embs1[13], mark_embs1[0], get_positive_feats(mkbls1, 13, 0))

Variable containing:
1.00000e-02 *
 -6.5129
[torch.FloatTensor of size 1x1]

**Deliverable 4.4**
Implement `score_instance()` and `instance_top_scores()` in `neural_net.SequentialScorer`. Their purpose is the same as the one in `FFCoref`, but they require the extra embeddings parameter. The former will require some changes to adapt to the different `forward()`, but the latter can be identical to its correlate in `FFCoref` if implemented correctly.
(1 point)

* **Test:** `tests\test_neural_coref.py:test_sequential_scorer_score_instance_d4_4(), test_sequential_scorer_instance_top_scores_d4_4()`

In [75]:
scorer.score_instance(mark_embs1, all_markables[1], 13, coref_features.minimal_features)

Variable containing:

Columns 0 to 9 
1.00000e-02 *
 -6.5129 -3.6650 -2.7447 -4.9805 -3.6708 -3.9170 -3.0245 -2.2054 -3.0073 -2.2837

Columns 10 to 13 
1.00000e-02 *
 -2.4808 -5.2409 -1.9311 -1.6140
[torch.FloatTensor of size 1x14]

Due to the length of our documents and number of parameters, torch may not work properly on the entire text. We'll truncate the files before we train and use only the minimal pairwise feature space, causing our performance to be suboptimal.

In [76]:
torch.manual_seed(1984)
tr_word_lstm = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.2)
tr_attn_layer = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
tr_scorer = neural_net.SequentialScorer(WORD_LSTM_EMB_DIM, min_features, BOOLEAN_FEATURE_DIM, SCORER_HIDDEN_DIM)
optimizer = optim.SGD(list(tr_word_lstm.parameters()) + list(tr_attn_layer.parameters()) + list(tr_scorer.parameters()), lr=ETA_0)
neural_net.train(tr_word_lstm, tr_attn_layer, tr_scorer,\
                 optimizer, all_words, all_markables, coref_features.minimal_features, word_limit=150, epochs=3)

Epoch 31 complete.
Document losses = 0.93932, 0.62971, 1.57278, 0.91849, 0.89681, 0.99915, 0.93578, 0.59647, 0.47273, 0.96949, 0.80176, 0.81332, 0.90159, 0.91200, 0.74807, 0.68139, 0.50189, 0.59706, 0.90399, 0.27876, 1.22716, 1.08047
Overall loss = 0.83621
Epoch 31 complete.
Document losses = 0.95553, 0.89429, 0.66035, 0.46930, 0.78851, 0.93100, 0.84361, 0.83657, 0.39137, 0.71947, 0.78565, 1.10904, 0.57058, 1.02704, 0.81013, 0.43853, 0.40327, 0.76322, 0.62335, 1.40899, 0.73699, 1.09462
Overall loss = 0.77141
Epoch 31 complete.
Document losses = 0.69691, 0.31914, 1.00857, 0.64840, 0.77790, 0.90615, 0.81879, 0.94573, 0.48738, 0.63369, 0.66208, 0.62734, 0.56925, 0.57159, 0.65879, 0.43431, 0.65710, 0.76916, 0.47507, 0.22506, 0.84899, 1.04005
Overall loss = 0.66926


Let's evaluate on the entire dataset.

In [77]:
tr_resolver = neural_net.evaluate(tr_word_lstm, tr_attn_layer, tr_scorer, all_words, all_markables, coref_features.minimal_features)

F: 0.5207	R: 0.3944	P:0.7658


In [79]:
coref_metrics(tr_resolver, all_markables);

B-Cubed: 0.5756	CEAF: 0.4551	MUC: 0.5605	Average: 0.5304


In [78]:
dv_resolver = neural_net.evaluate(tr_word_lstm, tr_attn_layer, tr_scorer, all_words_dev, all_markables_dev, coref_features.minimal_features)

F: 0.5403	R: 0.3826	P:0.9194


In [80]:
coref_metrics(dv_resolver, all_markables_dev);

B-Cubed: 0.5653	CEAF: 0.3966	MUC: 0.5403	Average: 0.5007


In [81]:
# students can't run this
te_resolver = neural_net.evaluate(tr_word_lstm, tr_attn_layer, tr_scorer, all_words_test, all_markables_te, coref_features.minimal_features)
coref_metrics(te_resolver, all_markables_te);

F: 0.4630	R: 0.3378	P:0.7353
B-Cubed: 0.5334	CEAF: 0.4225	MUC: 0.4778	Average: 0.4779


## Pretrained Word Embeddings

**Deliverable 4.5**
Implement `utils.initialize_with_pretrained()`. Start by copying from your implementation in PS3.
(0.5 points)

* **Test:** `tests\test_neural_coref.test_pretrain_embeddings_d4_5()`

**Note** that there is a new pretrained file in the `data` folder. Although from the same original source, it is trimmed to the vocabulary in our new dataset, so don't use the same file from PS3. In addition to this attribute, it includes a special token called **&lt;UNK&gt;**. You should use its assigned vector to initialize vectors for all unknown words in the dataset.

Later, if you're interested in improving your model's performance, you may want to know that there are more special tokens with trained vectors in the data file:
* **&lt;S&gt;** signifies the beginning of a sentence.
* **&lt;/S&gt;** signifies the end of a sentence.
* **&lt;PAD&gt;** is used to pad short sentences (this one probably won't be useful).

In [82]:
pret_embs = pickle.load(open('data/pretrained-embeds-coref.pkl', 'rb'))

In [83]:
print(pret_embs['Fujitsu'][:5])

[ 0.34177107 -0.1528549  -0.06971747  0.12536518  0.31670848]


In [84]:
print(pret_embs['<UNK>'][:5])

[-0.2400684   0.0170402  -0.5328812   0.16161029 -0.03450763]


In [85]:
torch.manual_seed(1984)
tr_pt_word_lstm = neural_net.BiLSTMWordEmbedding(word_to_ix, WORD_EMB_DIM, WORD_LSTM_EMB_DIM, 1, 0.2)
utils.initialize_with_pretrained(pret_embs, tr_pt_word_lstm)
tr_pt_attn_layer = neural_net.AttentionBasedMarkableEmbedding(WORD_LSTM_EMB_DIM)
tr_pt_scorer = neural_net.SequentialScorer(WORD_LSTM_EMB_DIM, min_features, BOOLEAN_FEATURE_DIM, SCORER_HIDDEN_DIM)
optimizer = optim.SGD(list(tr_pt_word_lstm.parameters()) + list(tr_pt_attn_layer.parameters()) + list(tr_pt_scorer.parameters()), lr=ETA_0)
neural_net.train(tr_pt_word_lstm, tr_pt_attn_layer, tr_pt_scorer,\
                 optimizer, all_words, all_markables, coref_features.minimal_features, word_limit=150, epochs=5)

Epoch 31 complete.
Document losses = 0.93080, 0.65143, 1.54045, 0.94181, 0.92525, 0.99127, 0.95597, 0.78670, 0.58961, 1.09405, 0.64312, 0.75519, 0.83328, 0.82583, 0.49011, 0.64926, 0.60697, 0.60192, 0.76932, 0.31302, 1.40232, 1.03551
Overall loss = 0.83530
Epoch 31 complete.
Document losses = 0.97924, 0.95806, 0.87604, 0.71278, 1.11671, 1.07097, 0.93572, 0.83217, 0.40688, 0.77817, 0.62955, 0.56910, 0.63388, 0.80036, 0.45497, 0.65841, 0.79375, 0.69298, 0.77241, 0.18915, 1.58285, 1.09280
Overall loss = 0.80113
Epoch 31 complete.
Document losses = 1.00449, 0.87274, 0.61690, 0.45452, 0.78579, 0.86951, 0.79216, 0.57742, 0.55570, 0.53367, 0.76431, 0.51676, 0.55175, 0.94359, 0.84593, 0.43401, 0.36180, 0.78888, 0.64513, 1.27888, 0.68404, 1.03125
Overall loss = 0.71666
Epoch 31 complete.
Document losses = 0.85349, 0.21191, 1.25798, 0.74668, 1.24524, 1.03969, 0.88672, 0.67221, 0.36734, 0.55026, 0.64822, 0.66793, 0.58037, 0.60304, 0.71610, 0.44662, 0.64840, 0.82709, 0.51960, 0.18410, 0.88014, 1.0

In [86]:
tr_pt_resolver = neural_net.evaluate(tr_pt_word_lstm, tr_pt_attn_layer, tr_pt_scorer, all_words, all_markables, coref_features.minimal_features)

F: 0.5253	R: 0.3979	P:0.7725


In [87]:
coref_metrics(tr_pt_resolver, all_markables);

B-Cubed: 0.5756	CEAF: 0.4551	MUC: 0.5605	Average: 0.5304


In [88]:
tr_pt_resolver_dev = neural_net.evaluate(tr_pt_word_lstm, tr_pt_attn_layer, tr_pt_scorer, all_words_dev, all_markables_dev, coref_features.minimal_features)
coref.write_predictions(tr_pt_resolver_dev,
                        all_markables_dev,
                        'predictions/nn-dev.preds');
coref_metrics(tr_pt_resolver_dev, all_markables_dev);

F: 0.5308	R: 0.3758	P:0.9032
B-Cubed: 0.5653	CEAF: 0.3966	MUC: 0.5403	Average: 0.5007


In [89]:
# students can't run this
tr_pt_resolver_te = neural_net.evaluate(tr_pt_word_lstm, tr_pt_attn_layer, tr_pt_scorer, all_words_test, all_markables_te, coref_features.minimal_features)
coref.write_predictions(tr_pt_resolver_te,
                        all_markables_te,
                        'predictions/nn-test.preds');
coref_metrics(tr_pt_resolver_te, all_markables_te);

F: 0.4556	R: 0.3324	P:0.7235
B-Cubed: 0.5334	CEAF: 0.4225	MUC: 0.4778	Average: 0.4779


**Deliverable 4.6** (7650 only; 4650 optional)

To match nominals, it is often necessary to capture **semantics**. Find a paper (in ACL, NAACL, EACL, EMNLP, or TACL, since 2007) that attempts to use semantic analysis to do nominal coreference, and explain:

- What form of semantics they are trying to capture (e.g., synonymy, hypernymy, predicate-argument, distributional)
- How they formalize semantics into features, constraints, or some other preference
- How much it helps

Put your answer in `text-answers.md`.
(1 point)

As usual, if you are in 4650 and you do this problem, you will be graded on the 7650 rubric.

## Part 5: Domain Adaptation (no deliverables)

Our dataset has a second part, which is a corpus of fairy tales, rather than news stories.

Let's take advantage of this setup to see how well our WSJ-trained model does over the fairy tale data, as opposed to a model trained on the fairy tales themselves.

In [90]:
ft_dv_dir = os.path.join('data','tales','dev')
ft_tr_dir = os.path.join('data','tales','train')
ft_te_dir = os.path.join('data','tales','test')

In [91]:
ft_all_markables, ft_all_words = coref.read_dataset(ft_tr_dir, tagger=pos_tag)
ft_all_markables_dev, ft_all_words_dev = coref.read_dataset(ft_dv_dir, tagger=pos_tag)
ft_all_markables_te, ft_all_words_te = coref.read_dataset(ft_te_dir, tagger=pos_tag)

In [92]:
coref.eval_on_dataset(exact_matcher, ft_all_markables);

F: 0.6511	R: 0.5545	P:0.7884


The exact matcher is getting much higher numbers on this dataset than on WSJ. Let's train an ML system and see the differences. We'll use `FFCoref` from section 3.

In [93]:
torch.manual_seed(1984) # DO NOT CHANGE
coref_ff_fairy = coref_learning.FFCoref(min_features_and_distances, 50)
optimizer = optim.SGD(coref_ff_fairy.parameters(), lr=ETA_0)
coref_learning.train(coref_ff_fairy, optimizer, ft_all_markables, joint_feats, epochs=2)

Loss = 0.9428621476767014
Loss = 0.9798036302839007


In [94]:
# in-domain trained, tested on fairy tale data
coref_learning.evaluate(coref_ff_fairy, ft_all_markables_dev, joint_feats)

F: 0.5583	R: 0.4747	P:0.6776


In [95]:
ft_ff_matcher = coref_learning.make_resolver(joint_feats, coref_ff_fairy)
coref_metrics(ft_ff_matcher, ft_all_markables_dev);

B-Cubed: 0.4673	CEAF: 0.3698	MUC: 0.6070	Average: 0.4814


These are the **in-domain** dev set scores for the fairy tale dataset.

Recall that the in-domain numbers for the WSJ portion were the following:

In [96]:
coref_metrics(ff_w_dist_matcher, all_markables_dev);

B-Cubed: 0.5641	CEAF: 0.3940	MUC: 0.5308	Average: 0.4963


Now we can see how the **cross-domain** results look. Can a model trained on news data be reliably used in a fairy-tale setting? How about the converse?

In [97]:
# trained on fairy, tested on WSJ
coref_metrics(ft_ff_matcher, all_markables_dev);

B-Cubed: 0.5329	CEAF: 0.4101	MUC: 0.4692	Average: 0.4707


In [98]:
# trained on WSJ, tested on fairy
coref_metrics(ff_w_dist_matcher, ft_all_markables_dev);

B-Cubed: 0.4615	CEAF: 0.3590	MUC: 0.6122	Average: 0.4776


These are lower than in-domain, but surprisingly enough not too low. In other tasks this problem is more acute.

If you're interested, you may try and join the two training sets together and see where that gets you. You can also use this extra data (only training sets!) for your bakeoff.

# 6. Final bakeoff!

This is for extra credit only. Ideas for improvements:

- Cost-sensitive training to balance precision and recall
- Syntax (you can parse all the markables as a preprocessing step)
  - Tree distance
  - Syntactic parallelism
  - Better head matching
- Add layers
- Better learning
- Ensemble (average) multiple models together

Feel free to search the research literature (via Google scholar) to get ideas. If you use an idea from another paper, mention the paper (authors, title, and URL) in your comments in `coref_features.py`.

As usual, sometimes improvement can also come from tweaking parameters in the neural nets.

In section 4, recall that we truncated the training documents - maybe more data (or different portions of each document) can help.

To use cuda, pass in use_cuda=True into `neural_net.train()` and `utils.initialize_with_pretrained()`.

**Deliverable 6**. Copy the applicable cells from section 3 or 4 by your choice to output predictions for both the dev and test sets.

Write your predictions to `predictions/bakeoff-dev.preds` and `predictions/bakeoff-test.preds`.

Due to the complex evaluation structure, this will be done from the tarball and not on Kaggle.

Scoring:

Test set F1 must be greater than .51 to get any extra credit.

- Dev F1 > .535: +0.5 points
- Dev F1 > .55: +1 points
- Dev F1 > .6, Test F1 > .55: +3 points
- Best in 4650: +0.5 points
- Best in 7650: +0.5 points
- Better than best TA/prof system: +0.5 points